In [1]:
import pandas as pd
import matplotlib as plt
from utils import transform_bbch_data, count_daily_measurements, clean_timestamp, find_common_dates, calculate_cumulative_gdd, calculate_accumulated_nitrates, calculate_cumulative_par, daily_average_humidity, merge_variables

### <b> Dataset Creation </b>

* Goal: Create a dataset based on lettuce plants' biometry and environmental variables for a greenhouse experiment

#### <b> Biometry variables </b>

* <b> Diameter </b>: The diameter of a chosen leaf of the plant, expressed in centimeters (cm)
* <b> Perpendicular </b>: The length of perpendicular line to the chosen diameter of a chosen leaf, expressed in centimeters (cm)
* <b> Weight </b>: The weight of the plant, expressed in kilograms (Kg)
* <b> Height </b>: The height of the plant, expressed in centimeters (cm)
* <b> Thickness </b>: Leaf thickness, expressed in centimeters (cm)
* <b> Number of leaves (N leaves) </b>: The number of leaves the plant presents 

#### <b> Environmental variables </b>

* <b> Irrigation </b>
* <b> Humidity </b>
* <b> Temperature </b>
* <b> PAR radiation </b>


### <b> Biometry </b>

In [2]:
# Previously created biometry dataset
biometry = pd.read_csv("data/final_biometric_data.csv")
biometry.rename(columns={"Unnamed: 0":"Date", "Sample":"Sample?", "CODE":"Sample"}, inplace=True)
biometry.drop(columns=['Max. Temp.', 'Min. Temp.', 'Mean. Temp.',
       'Max. Hum.', 'Min. Hum.', 'Mean. Hum.', 'Combined Temperature Average',
       'Combined Temperature Std Dev', 'Combined Humidity Average',
       'Combined Humidity Std Dev','Sample?','BBCH'], inplace=True)

In [3]:
bbch = pd.read_excel('raw_data/biom_to_BBCH.xlsx')
bbch.drop(columns={'Number', 'Line', 'Sample'}, inplace=True)

In [4]:
bbch_cols = list(bbch.columns)

In [5]:
new_bbch = transform_bbch_data(bbch)

In [6]:
new_bbch.rename(columns={'CODE':'Sample'}, inplace=True)

In [7]:
new_bbch

,Sample,Date,BBCH
0,RNGRA1,2024-09-07,19
1,RNGRA1,2024-09-11,19
2,RNGRA1,2024-09-14,19
3,RNGRA1,2024-09-17,37
4,RNGRA1,2024-09-20,37
...,...,...,...
355,RWGRC5,2024-09-17,19
356,RWGRC5,2024-09-20,19
357,RWGRC5,2024-09-24,19
358,RWGRC5,2024-09-29,19


In [8]:
# Set Date column to datetime type
biometry['Date'] = pd.to_datetime(biometry['Date'])

<b> Total number of plants </b>

In [9]:
biometry['Sample'].nunique()

45

* There's 45 plants in this experiment, each represented by its unique code, represented in the CODE column

<b> Dates where biometry was measured </b>

In [10]:
biometry['Date'].nunique()

8

* Measurements of the biometry of the plants were taken at 8 different days

### <b> Irrigation </b>

In [11]:
irrigation = pd.read_excel("raw_data/irrigation.xlsx")

In [12]:
# Set Date column to datetime type
irrigation['Date'] = pd.to_datetime(irrigation['Date'])

In [13]:
irrigation

,Date,Sample,Quantity (mL)
0,2024-08-24,rngra1,50.0
1,2024-08-24,rngra2,50.0
2,2024-08-24,rngra3,50.0
3,2024-08-24,rngra4,50.0
4,2024-08-24,rngra5,50.0
...,...,...,...
985,2024-10-01,rwgrc1,7.5
986,2024-10-01,rwgrc2,14.5
987,2024-10-01,rwgrc3,5.0
988,2024-10-01,rwgrc4,12.0


<b> Number of irrigation days </b>

In [14]:
irrigation['Date'].nunique()

23

* There's 23 days of irrigation

### <b> Temperature </b>

In [15]:
temperature = pd.read_excel("raw_data/inside_2.xlsx")[['date','temperature']]

In [16]:
temperature.rename(columns={'date':'Date'}, inplace=True)

In [17]:
temperature

,Date,temperature
0,2024-08-22,18.200000
1,2024-08-22,17.200000
2,2024-08-22,17.300000
3,2024-08-22,17.700000
4,2024-08-22,17.400000
...,...,...
941,2024-10-03,20.703388
942,2024-10-03,21.296177
943,2024-10-03,21.758629
944,2024-10-03,21.531219


In [18]:
# Set Date column to datetime type
temperature['Date'] = pd.to_datetime(temperature['Date'])

In [19]:
temperature

,Date,temperature
0,2024-08-22,18.200000
1,2024-08-22,17.200000
2,2024-08-22,17.300000
3,2024-08-22,17.700000
4,2024-08-22,17.400000
...,...,...
941,2024-10-03,20.703388
942,2024-10-03,21.296177
943,2024-10-03,21.758629
944,2024-10-03,21.531219


In [20]:
temperature['Date'].nunique()

43

* Measurements of the temperature of the greenhouse were taken at 43 different days

In [21]:
# Daily number of temperature measurements
temp_date_dict = count_daily_measurements(temperature)

In [22]:
temp_date_dict

{Timestamp('2024-08-22 00:00:00'): 25,
 Timestamp('2024-08-23 00:00:00'): 24,
 Timestamp('2024-08-24 00:00:00'): 24,
 Timestamp('2024-08-25 00:00:00'): 24,
 Timestamp('2024-08-26 00:00:00'): 24,
 Timestamp('2024-08-27 00:00:00'): 24,
 Timestamp('2024-08-28 00:00:00'): 24,
 Timestamp('2024-08-29 00:00:00'): 24,
 Timestamp('2024-08-30 00:00:00'): 24,
 Timestamp('2024-08-31 00:00:00'): 24,
 Timestamp('2024-09-01 00:00:00'): 23,
 Timestamp('2024-09-02 00:00:00'): 24,
 Timestamp('2024-09-03 00:00:00'): 24,
 Timestamp('2024-09-04 00:00:00'): 24,
 Timestamp('2024-09-05 00:00:00'): 24,
 Timestamp('2024-09-06 00:00:00'): 24,
 Timestamp('2024-09-07 00:00:00'): 24,
 Timestamp('2024-09-08 00:00:00'): 24,
 Timestamp('2024-09-09 00:00:00'): 24,
 Timestamp('2024-09-10 00:00:00'): 24,
 Timestamp('2024-09-11 00:00:00'): 24,
 Timestamp('2024-09-12 00:00:00'): 19,
 Timestamp('2024-09-13 00:00:00'): 18,
 Timestamp('2024-09-14 00:00:00'): 24,
 Timestamp('2024-09-15 00:00:00'): 24,
 Timestamp('2024-09-16 00

In [23]:
temp_date_dict.values()

dict_values([25, 24, 24, 24, 24, 24, 24, 24, 24, 24, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 19, 18, 24, 24, 16, 9, 14, 21, 20, 24, 14, 13, 23, 24, 24, 22, 24, 23, 23, 24, 24, 15])

* As we can tell by the values of the date_dict dictionary the number of temperature measurements wasn't constant during the experiment.

### <b> Humidity </b>

In [24]:
humidity = pd.read_excel("raw_data/inside_2.xlsx")[['date','humidity']]
humidity.rename(columns={'date':'Date'}, inplace=True)

In [25]:
# Set Date column to datetime type
humidity['Date'] = pd.to_datetime(humidity['Date'])

In [26]:
humidity['Date'].nunique()

43

In [27]:
humidity

,Date,humidity
0,2024-08-22,66.100000
1,2024-08-22,70.700000
2,2024-08-22,74.000000
3,2024-08-22,76.600000
4,2024-08-22,77.800000
...,...,...
941,2024-10-03,86.559822
942,2024-10-03,85.557480
943,2024-10-03,84.059044
944,2024-10-03,82.733772


In [28]:
hum_date_dict = count_daily_measurements(humidity)

In [29]:
hum_date_dict.values()

dict_values([25, 24, 24, 24, 24, 24, 24, 24, 24, 24, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 19, 18, 24, 24, 16, 9, 14, 21, 20, 24, 14, 13, 23, 24, 24, 22, 24, 23, 23, 24, 24, 15])

* The number of humidity measurements follows the number of temperature measurements (captured by the same sensors)

### <b> PAR </b>

In [30]:
par = pd.read_excel('raw_data/PAR.xlsx')

# Set Date column to datetime type using clean_timestamp from utils.py
par['Date'] = par['Date'].apply(clean_timestamp)

In [31]:
par['Date'].nunique()

19

In [32]:
par_date_dict = count_daily_measurements(par)

In [33]:
par_date_dict

{Timestamp('2024-09-02 00:00:00'): 5,
 Timestamp('2024-09-03 00:00:00'): 24,
 Timestamp('2024-09-04 00:00:00'): 24,
 Timestamp('2024-09-05 00:00:00'): 24,
 Timestamp('2024-09-06 00:00:00'): 24,
 Timestamp('2024-09-07 00:00:00'): 23,
 Timestamp('2024-09-08 00:00:00'): 24,
 Timestamp('2024-09-09 00:00:00'): 24,
 Timestamp('2024-09-10 00:00:00'): 24,
 Timestamp('2024-09-11 00:00:00'): 24,
 Timestamp('2024-09-12 00:00:00'): 24,
 Timestamp('2024-09-13 00:00:00'): 25,
 Timestamp('2024-09-14 00:00:00'): 23,
 Timestamp('2024-09-15 00:00:00'): 24,
 Timestamp('2024-09-16 00:00:00'): 23,
 Timestamp('2024-09-17 00:00:00'): 24,
 Timestamp('2024-09-18 00:00:00'): 24,
 Timestamp('2024-09-19 00:00:00'): 24,
 Timestamp('2024-09-20 00:00:00'): 10}

<b> Common Dates amongst biometric and environmental dataframes </b>

In [34]:
dataframes = {'Biometry':biometry, 'Irrigation':irrigation, 'Humidity':humidity,'Temperature':temperature, 'Par':par}

In [35]:
common_dates = find_common_dates(dataframes, check_num = 1)

Biometry x Irrigation | No. Common Dates: 4

Biometry x Humidity | No. Common Dates: 8

Biometry x Temperature | No. Common Dates: 8

Biometry x Par | No. Common Dates: 5

Irrigation x Humidity | No. Common Dates: 23

Irrigation x Temperature | No. Common Dates: 23

Irrigation x Par | No. Common Dates: 11

Humidity x Temperature | No. Common Dates: 43

Humidity x Par | No. Common Dates: 19

Temperature x Par | No. Common Dates: 19



In [36]:
common_dates

{'Biometry x Irrigation': {Timestamp('2024-09-07 00:00:00'),
  Timestamp('2024-09-11 00:00:00'),
  Timestamp('2024-09-17 00:00:00'),
  Timestamp('2024-09-20 00:00:00')},
 'Biometry x Humidity': {Timestamp('2024-09-07 00:00:00'),
  Timestamp('2024-09-11 00:00:00'),
  Timestamp('2024-09-14 00:00:00'),
  Timestamp('2024-09-17 00:00:00'),
  Timestamp('2024-09-20 00:00:00'),
  Timestamp('2024-09-24 00:00:00'),
  Timestamp('2024-09-29 00:00:00'),
  Timestamp('2024-10-03 00:00:00')},
 'Biometry x Temperature': {Timestamp('2024-09-07 00:00:00'),
  Timestamp('2024-09-11 00:00:00'),
  Timestamp('2024-09-14 00:00:00'),
  Timestamp('2024-09-17 00:00:00'),
  Timestamp('2024-09-20 00:00:00'),
  Timestamp('2024-09-24 00:00:00'),
  Timestamp('2024-09-29 00:00:00'),
  Timestamp('2024-10-03 00:00:00')},
 'Biometry x Par': {Timestamp('2024-09-07 00:00:00'),
  Timestamp('2024-09-11 00:00:00'),
  Timestamp('2024-09-14 00:00:00'),
  Timestamp('2024-09-17 00:00:00'),
  Timestamp('2024-09-20 00:00:00')},
 'Ir

* Since there aren't many common dates amongst the biometry dataframes and environmental variables, we'll make use of the available environmental data to calculate cumulative values where possible to gather valuable information and append it to the corresponding biometry entries in the dataframe.

#### <b> Environmental Variables </b>

In [37]:
cumul_gdd = calculate_cumulative_gdd(temperature_df=temperature)

In [38]:
cumul_gdd

,Date,GDD Cumul.
0,2024-08-22,18.750000
1,2024-08-22,18.750000
2,2024-08-22,18.750000
3,2024-08-22,18.750000
4,2024-08-22,18.750000
...,...,...
941,2024-10-03,809.244245
942,2024-10-03,809.244245
943,2024-10-03,809.244245
944,2024-10-03,809.244245


<b> Accumulated PAR </b>

In [39]:
par_cumul = calculate_cumulative_par(par_df=par)

<b> Accumulated Solution and Nitrate Irrigation </b>

In [40]:
irr_df = calculate_accumulated_nitrates(irrigation_df=irrigation)

In [41]:
irr_df

,Date,Sample,Cumul. Irrigation (mL),Cumul. Nitrates (milimoles)
0,2024-08-24,RNGRA1,50.0,0.3000
1,2024-08-24,RNGRA2,50.0,0.3000
2,2024-08-24,RNGRA3,50.0,0.3000
3,2024-08-24,RNGRA4,50.0,0.3000
4,2024-08-24,RNGRA5,50.0,0.3000
...,...,...,...,...
985,2024-10-01,RWGRC1,1084.0,18.4280
986,2024-10-01,RWGRC2,877.5,14.9175
987,2024-10-01,RWGRC3,1045.5,17.7735
988,2024-10-01,RWGRC4,1058.0,17.9860


<b> Average daily Humidity </b>

In [42]:
hum_avg = daily_average_humidity(humidity_df=humidity)

<b> Combine all dataframes </b>

In [43]:
merged_df = merge_variables(bbch_df=new_bbch, biometry_df=biometry, gdd_df=cumul_gdd, par_df=par_cumul, nitrates_df=irr_df)

In [44]:
par_cumul

,Date,PAR Cumul.
0,2024-09-02,87.0
1,2024-09-03,1860.3
2,2024-09-04,3788.5
3,2024-09-05,5441.0
4,2024-09-06,6577.0
5,2024-09-07,8284.7
6,2024-09-08,10229.2
7,2024-09-09,12051.7
8,2024-09-10,14750.7
9,2024-09-11,16804.0


In [45]:
merged_df[merged_df['Sample'] == 'RNGRA3']

,Date,Number,Line,Sample,No leaves,Diameter,Perpendicular,Height,Average Leaf Thickness,BBCH,GDD Cumul.,PAR Cumul.,Cumul. Irrigation (mL),Cumul. Nitrates (milimoles)
2,2024-09-07,16,3,RNGRA3,9,9.8,9.7,10.70,0.000000,19,317.050000,8284.7,350.00,2.1000
47,2024-09-11,16,3,RNGRA3,12,14.3,12.3,12.80,0.000000,19,391.750000,16804.0,462.25,2.7735
92,2024-09-14,16,3,RNGRA3,12,21.2,13.8,12.50,0.000000,19,457.100000,22970.1,530.50,3.1830
137,2024-09-17,16,3,RNGRA3,14,21.5,16.8,11.30,0.000000,19,529.378430,26761.5,661.50,3.9690
182,2024-09-20,16,3,RNGRA3,14,21.5,20.4,11.90,0.416667,19,589.001102,28775.3,737.50,4.4250
227,2024-09-24,16,3,RNGRA3,14,28.5,25.7,13.55,0.293333,19,660.469683,28775.3,862.50,5.1750
272,2024-09-29,16,3,RNGRA3,13,27.8,28.5,13.05,0.430000,19,743.211838,28775.3,997.50,5.9850
317,2024-10-03,16,3,RNGRA3,16,31.4,30.6,11.45,0.264000,19,809.244245,28775.3,1141.00,6.8460


In [46]:
merged_df.to_csv('03_biometric.csv')

In [47]:
merged_df.columns

Index(['Date', 'Number', 'Line', 'Sample', 'No leaves', 'Diameter',
       'Perpendicular', 'Height', 'Average Leaf Thickness', 'BBCH',
       'GDD Cumul.', 'PAR Cumul.', 'Cumul. Irrigation (mL)',
       'Cumul. Nitrates (milimoles)'],
      dtype='object')

### <b> EDA </b>

### <b> Models </b>

In [48]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [49]:
# Custom function for splitting and scaling the data
def custom_train_test_split(df, label):
    X = df.drop(columns=label)
    y = df[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled, y_train, y_test

In [50]:
def predict_all_biometry_variables(df, biometry_columns):
    results = []
    for target in biometry_columns:
        X_train, X_test, y_train, y_test = custom_train_test_split(df, label=target)
        
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Target Variable': target,
            'MSE': mse,
            'R^2': r2
        })
    
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df

In [51]:
biometry_columns = ['No leaves', 'Diameter', 'Perpendicular', 'Height', 'Average Leaf Thickness']
results_0 = predict_all_biometry_variables(merged_df.drop(columns={'Date','Sample'}), biometry_columns)

          Target Variable       MSE       R^2
0               No leaves  4.095549  0.540597
1                Diameter  3.653804  0.892059
2           Perpendicular  4.109736  0.885233
3                  Height  2.372677  0.462147
4  Average Leaf Thickness  0.006390  0.767298


In [52]:
from sklearn.linear_model import Ridge, Lasso

def predict_with_regularization(df, biometry_columns, model_type='ridge'):
    results = []
    for target in biometry_columns:
        X_train, X_test, y_train, y_test = custom_train_test_split(df, label=target)
        
        if model_type == 'ridge':
            model = Ridge(alpha=1.0)
        elif model_type == 'lasso':
            model = Lasso(alpha=0.01)
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Target Variable': target,
            'MSE': mse,
            'R^2': r2
        })
    
    return pd.DataFrame(results)

In [53]:
results_ridge = predict_with_regularization(merged_df.drop(columns={'Date','Sample'}), biometry_columns, 'ridge')
results_ridge

,Target Variable,MSE,R^2
0,No leaves,4.092600,0.540927
1,Diameter,3.630353,0.892752
2,Perpendicular,4.084410,0.885940
3,Height,2.375665,0.461470
4,Average Leaf Thickness,0.006322,0.769771


In [54]:
results_lasso = predict_with_regularization(merged_df.drop(columns={'Date','Sample'}), biometry_columns, 'lasso')
results_lasso

,Target Variable,MSE,R^2
0,No leaves,4.083737,0.541922
1,Diameter,3.646586,0.892272
2,Perpendicular,4.074583,0.886215
3,Height,2.387382,0.458814
4,Average Leaf Thickness,0.006084,0.778470


In [55]:
from sklearn.ensemble import RandomForestRegressor

def predict_with_random_forest(df, biometry_columns):
    results = []
    for target in biometry_columns:
        X_train, X_test, y_train, y_test = custom_train_test_split(df, label=target)
        
        model = RandomForestRegressor(random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Target Variable': target,
            'MSE': mse,
            'R^2': r2
        })
    
    return pd.DataFrame(results)

In [56]:
results_rf = predict_with_random_forest(merged_df.drop(columns={'Date','Sample'}), biometry_columns)

In [57]:
results_rf

,Target Variable,MSE,R^2
0,No leaves,2.460717,0.723978
1,Diameter,3.996564,0.881933
2,Perpendicular,4.416743,0.876660
3,Height,2.136601,0.515662
4,Average Leaf Thickness,0.002060,0.924975


In [58]:
from xgboost import XGBRegressor

def predict_with_xgboost(df, biometry_columns):
    results = []
    for target in biometry_columns:
        X_train, X_test, y_train, y_test = custom_train_test_split(df, label=target)
        
        model = XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        results.append({
            'Target Variable': target,
            'MSE': mse,
            'R^2': r2
        })
    
    return pd.DataFrame(results)

In [59]:
results_xgb = predict_with_xgboost(merged_df.drop(columns={'Date','Sample'}), biometry_columns)
print("XGBoost Regression Results:")
print(results_xgb)

XGBoost Regression Results:
          Target Variable       MSE       R^2
0               No leaves  2.230678  0.749782
1                Diameter  4.811125  0.857869
2           Perpendicular  5.197446  0.854858
3                  Height  2.289854  0.480922
4  Average Leaf Thickness  0.003190  0.883834
